In [6]:
import requests

rpc = "https://muddy-bitter-dew.quiknode.pro/8d853a8106f75d700a1904f46105e55fc2a637ae"
etherscan_api_key = "M454MSIZNREPEV3YBPKY66792EUTYAB2C2"

def get_latest_block_number():
    payload = {
        "jsonrpc": "2.0",
        "method": "eth_blockNumber",
        "params": [],
        "id": 1
    }
    response = requests.post(rpc, json=payload).json()
    return int(response["result"], 16)  # Hex to decimal

print("Latest block:", get_latest_block_number())

def get_contract_abi(contract_address):
    """Get contract ABI from Etherscan API"""
    url = f"https://api.etherscan.io/api?module=contract&action=getabi&address={contract_address}&apikey={etherscan_api_key}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()["result"]
    except Exception as e:
        print(f"Error getting ABI: {e}")
        return None

print(get_contract_abi("0xBB9bc244D798123fDe783fCc1C72d3Bb8C189413"))



print(get_tx_by_address("0xBB9bc244D798123fDe783fCc1C72d3Bb8C189413"))


Latest block: 21809234
[{"constant":true,"inputs":[{"name":"","type":"uint256"}],"name":"proposals","outputs":[{"name":"recipient","type":"address"},{"name":"amount","type":"uint256"},{"name":"description","type":"string"},{"name":"votingDeadline","type":"uint256"},{"name":"open","type":"bool"},{"name":"proposalPassed","type":"bool"},{"name":"proposalHash","type":"bytes32"},{"name":"proposalDeposit","type":"uint256"},{"name":"newCurator","type":"bool"},{"name":"yea","type":"uint256"},{"name":"nay","type":"uint256"},{"name":"creator","type":"address"}],"type":"function"},{"constant":false,"inputs":[{"name":"_spender","type":"address"},{"name":"_amount","type":"uint256"}],"name":"approve","outputs":[{"name":"success","type":"bool"}],"type":"function"},{"constant":true,"inputs":[],"name":"minTokensToCreate","outputs":[{"name":"","type":"uint256"}],"type":"function"},{"constant":true,"inputs":[],"name":"rewardAccount","outputs":[{"name":"","type":"address"}],"type":"function"},{"constant":

In [17]:
import pandas as pd
def get_balance_eth(address):
    url = f"https://api.etherscan.io/api?module=account&action=balance&address={address}&tag=latest&apikey={etherscan_api_key}"

    eth = requests.get(url).json()["result"]
    eth = float(eth) / (10 ** 18)
    return eth

def get_tx_by_address(address):
    url = f"https://api.etherscan.io/api?module=account&action=txlist&address={address}&startblock=0&endblock=99999999&page=1&offset=5&sort=desc&apikey={etherscan_api_key}"
    txs = requests.get(url).json()["result"]
    
    # Create DataFrame
    df = pd.DataFrame(txs)
    
    # Convert numeric columns
    numeric_cols = ['blockNumber', 'timeStamp', 'value', 'gas', 'gasPrice', 'gasUsed']
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col])
    
    # Convert timestamp to datetime
    df['timeStamp'] = pd.to_datetime(df['timeStamp'], unit='s')
    
    # Convert Wei to ETH for value
    df['value_eth'] = df['value'].apply(lambda x: x / 1e18)
    
    # Select and reorder columns
    columns = ['timeStamp', 'from', 'to', 'value_eth', 'gasUsed', 'isError', 'hash']
    return df[columns]

def erc20_transfers(address):
    url = f"https://api.etherscan.io/api?module=account&action=tokentx&address={address}&startblock=0&endblock=99999999&page=1&offset=5&sort=desc&apikey={etherscan_api_key}"
    transfers = requests.get(url).json()["result"]
    
    # Create DataFrame
    df = pd.DataFrame(transfers)
    
    # Convert numeric columns
    numeric_cols = ['blockNumber', 'timeStamp', 'value', 'gas', 'gasPrice', 'gasUsed']
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col])
    
    # Convert timestamp to datetime
    df['timeStamp'] = pd.to_datetime(df['timeStamp'], unit='s')
    
    # Convert token value based on decimals
    df['token_value'] = df.apply(lambda row: int(row['value']) / (10 ** int(row['tokenDecimal'])) if row['tokenDecimal'] != '0' else int(row['value']), axis=1)
    
    # Select and reorder columns
    columns = ['timeStamp', 'from', 'to', 'tokenName', 'token_value', 'tokenSymbol', 'hash']
    return df[columns]

print("Latest 5 transactions:")
print(get_tx_by_address("0xBB9bc244D798123fDe783fCc1C72d3Bb8C189413"))
print("\nLatest 5 ERC20 transfers:")
print(erc20_transfers("0xBB9bc244D798123fDe783fCc1C72d3Bb8C189413"))


Latest 5 transactions:
            timeStamp                                        from  \
0 2025-02-08 15:47:23  0x4e05a4e4f2ce7ba5100ceeb3ae6c74a5226c822b   
1 2025-02-02 02:58:59  0x30355646e90b5913264132e1d8eaf7bdd5841d0a   
2 2025-02-02 02:56:23  0x30355646e90b5913264132e1d8eaf7bdd5841d0a   
3 2025-02-01 02:47:47  0xacdb137f14fdff6902cbbb7f0b69052718fbb83d   
4 2025-01-30 10:18:23  0x4bdcaf690eaa1d4eb973ca2c353747370a3016f5   

                                           to  value_eth  gasUsed isError  \
0  0xbb9bc244d798123fde783fcc1c72d3bb8c189413        0.0    26159       0   
1  0xbb9bc244d798123fde783fcc1c72d3bb8c189413        0.0    21940       1   
2  0xbb9bc244d798123fde783fcc1c72d3bb8c189413        0.0   209999       1   
3  0xbb9bc244d798123fde783fcc1c72d3bb8c189413        0.0    46348       0   
4  0xbb9bc244d798123fde783fcc1c72d3bb8c189413        0.0    21958       0   

                                                hash  
0  0xd8f87d0a88ca294c8af4fe4f2b680ac1618b964

ValueError: Integer out of range. at position 3

In [ ]:
def get_block_by_number(block_number):
    payload = {
        "jsonrpc": "2.0",
        "method": "eth_getBlockByNumber",
        "params": [hex(block_number), True],  # Hex string, include transactions
        "id": 1
    }
    response = requests.post(rpc, json=payload).json()
    return response["result"]

block_data = get_block_by_number(get_latest_block_number())
for item in block_data:
    print(item)
    print(block_data[item])
    

In [28]:
import csv

def save_transactions_to_csv(block_number, filename="transactions.csv"):
    block = get_block_by_number(block_number)
    with open(filename, "a", newline="") as f:
        writer = csv.writer(f)
        for tx in block["transactions"]:
            writer.writerow([
                tx["hash"],
                int(tx["gasPrice"], 16),
                int(tx["value"], 16),
                tx["from"]
            ])

save_transactions_to_csv(get_latest_block_number())

In [32]:
import csv

def rpc_request(method, params):
    payload = {
        "jsonrpc": "2.0",
        "method": method,
        "params": params,
        "id": 1
    }
    response = requests.post(rpc, json=payload).json()
    return response["result"]

def save_transactions_to_csv(block_number, filename="transactions.csv"):
    block = get_block_by_number(block_number)
    with open(filename, "a", newline="") as f:
        writer = csv.writer(f)
        for tx in block["transactions"]:
            # Get transaction receipt for status and gas used
            receipt = rpc_request("eth_getTransactionReceipt", [tx["hash"]])
            
            # Transaction Type Detection
            tx_type = "transfer"
            input_data = tx.get("input", "0x")
            contract_interaction = False
            
            if tx["to"] is None:
                tx_type = "contract_creation"
            elif input_data != "0x":
                tx_type = "contract_call"
                contract_interaction = True

            # ERC20/721 Transfer Detection (check logs)
            is_token_transfer = any(
                log["topics"][0] == "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"
                for log in receipt.get("logs", [])
            )

            # Write extended data
            writer.writerow([
                # Basic Info
                block_number,
                int(block["timestamp"], 16),  # Unix timestamp
                tx["hash"],
                tx["transactionIndex"],
                
                # Transaction Type
                tx_type,
                "token_transfer" if is_token_transfer else "",
                
                # Address Info
                tx.get("from", ""),
                tx.get("to", ""),
                receipt.get("contractAddress", ""),  # For contract creations
                
                # Value & Gas
                int(tx["value"], 16),
                int(tx["gasPrice"], 16),
                int(receipt.get("gasUsed", "0x0"), 16),
                
                # Contract Interaction Details
                contract_interaction,
                len(input_data),  # Input data size
                input_data[:10],  # Partial input (function selector)
                
                # Status
                "success" if int(receipt.get("status", "0x0"), 16) == 1 else "failed"
            ])

save_transactions_to_csv(get_latest_block_number()-10)

In [13]:
def analyze_failed_txs(block_number):
    block = get_block_by_number(block_number)
    failed = 0
    for tx_hash in block["transactions"]:
        receipt = requests.post(rpc, json={
            "jsonrpc": "2.0",
            "method": "eth_getTransactionReceipt",
            "params": [tx_hash],
            "id": 1
        }).json()["result"]
        
        if receipt and receipt["status"] == "0x0":
            failed += 1
    return failed / len(block["transactions"]) * 100  # Failure rate

def get_contract_storage(contract_address, position):
    return requests.post(rpc, json={
        "jsonrpc": "2.0",
        "method": "eth_getStorageAt",
        "params": [contract_address, hex(position), "latest"],
        "id": 1
    }).json()["result"]

def get_uniswap_swaps(pair_address):
    return requests.post(rpc, json={
        "jsonrpc": "2.0",
        "method": "eth_getLogs",
        "params": [{
            "fromBlock": "0x0",
            "address": pair_address,
            "topics": ["0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822"]
        }],
        "id": 1
    }).json()["result"]


def gas_forecast():
    history = requests.post(rpc, json={
        "jsonrpc": "2.0",
        "method": "eth_feeHistory",
        "params": ["0x4", "latest", [25, 50, 75]],
        "id": 1
    }).json()["result"]
    
    return {
        "base_fee_trend": [int(x, 16) for x in history["baseFeePerGas"]],
        "priority_percentiles": {
            "25": [int(x[0], 16) for x in history["reward"]],
            "50": [int(x[1], 16) for x in history["reward"]],
            "75": [int(x[2], 16) for x in history["reward"]]
        }
    }

In [ ]:
graph init \
  --protocol ethereum \
  --contract-name MyErc20Contract \
  --from-contract  \
  --network base \
  --index-events \
  myusername/test-subgraph \
  test-subgraph

In [ ]:
import subprocess

def init_subgraph_streaming(subgraph_name, directory, contract_address, network="mainnet", contract_name="USDC"):

    cmd = [
        "graph", "init",
        "--protocol", "ethereum",
        "--contract-name", contract_name,
        "--from-contract", contract_address,
        "--network", network,
        "--index-events",
        subgraph_name,
        directory
    ]

    # Start the process and redirect both stdout and stderr to the same stream.
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

    # Read and print output line by line as it is produced.
    while True:
        line = process.stdout.readline()
        if not line and process.poll() is not None:
            break
        if line:
            print(line.strip())

    return_code = process.wait()
    auth = ["graph", "auth", "19411d8c1971a3d6d465ec2fe6b4a6d8"]
    subprocess.run(auth)
    build = ["npm", "run", "deploy"]
    result = subprocess.run(build, cwd=directory,
                            input="0.2.0\n",  # Automatically provide the version label.
                            text=True, capture_output=True)

    # Optionally, print the deploy command's output.
    print(result.stdout)
    if result.stderr:
        print("Deploy stderr:", result.stderr)

    return return_code

# Example usage:
if __name__ == "__main__":
    subgraph_name = "baseallswaps"
    directory = "./theGraph/baseallswaps"
    contract_address = "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"  # USDC contract on Ethereum mainnet

    ret_code = init_subgraph_streaming(subgraph_name, directory, contract_address)
    if ret_code == 0:
        print("Subgraph initialized successfully.")
    else:
        print("Subgraph initialization encountered an error, return code:", ret_code)
